# IConfucius hacker guide to Odin.Fun

<img src="../images/confucius-hacker.webp" alt="Confucius" width="200">

We hacked into the Odin.Fun API using the following techniques:
- Install [IC inspector]() browser extension
- Study network requests in browser Inspect
- Use the AI build into the browser to create equivalent python code

Before you can run the JWT authenticated requests:
- Login to the IConfucius account from the browser
- Open Inspect
- Find the authenticated request in Network section
- Copy the JWT from the `Authorization` header
- Save it in the file: `scripts/.env` as:
  ```bash
  JWT=<the jwt token grabbed from /v1/token/{token_id}/comment endpoint>
  ```

Create a Conda environment:
  ```bash
  # install Miniconda on your system

  # create a conda environment
  conda create --name IConfucius python=3.11
  conda activate IConfucius

  # from IConfucius root folder
  pip install -r requirements.txt
  ```

==> Open the notebook and run it.


# Settings

In [15]:
# The IConfucius user ID, which is the same as it's principal ID
USER_ID = "qlnnh-6vcyp-27i57-mw6ya-r2hxi-kktfj-kyaer-26nds-64kyg-bfa2x-kae"

# The IConfucius token ID
TOKEN_ID = "29m8"

# Tell odin.fun who is calling the API
USER_AGENT = "IConfucius AI Agent"

# The Odin.Fun API endpoint
ODIN_FUN_API = "https://api.odin.fun"

## Verify we're in the Conda environment

In [16]:
import sys

print(sys.executable)

/opt/miniconda3/envs/IConfucius/bin/python


## Import python packages

In [17]:
import os
import sys
import json
import base64
import io

# from dotenv import load_dotenv
# import requests
import pprint
from pathlib import Path
import subprocess
import jupyter_black
import textwrap
import requests
import json
import jwt
from dotenv import load_dotenv
from get_jwt import get_jwt

import random

from run_llama_cpp import run_llama_cpp

# Activate the jupyter_black extension, which reformats code cells with black
# https://github.com/n8henrie/jupyter-black
jupyter_black.load()

# Load JWT from .env


**WARING WARNING WARNING**

The JWT provides full access to the IConfucius account.
- Do not share it with anyone.
- Do not print it, not even in this notebook, because you might check it into source control by accident

In [18]:
# Get JWT from the .env file
JWT = get_jwt("JWT")

# Get the IConfucius user data

This is an unauthenticated call, so all this data is public.

In [19]:
def get_user_data(user_id: str) -> dict:
    """
    Fetches user data from the API and returns the data as a Python dictionary.

    Returns:
        A Python dictionary representing the user data, or None if an error occurred.
    """

    url = f"{ODIN_FUN_API}/v1/user/{user_id}"
    headers = {
        "accept": "*/*",
        "accept-encoding": "gzip, deflate, br, zstd",
        "accept-language": "en-US,en;q=0.9",
        "origin": "",
        "referer": "",
        "user-agent": USER_AGENT,
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)

        # Check if the response is Brotli-compressed
        if response.headers.get("content-encoding") == "br":
            # Decompress the data using brotli.decompress() - already done by requests
            # decompressed_data = brotli.decompress(response.content)
            # decoded_data = decompressed_data.decode("utf-8")
            decoded_data = response.content.decode("utf-8")
        else:
            # Decode the decompressed data as UTF-8
            decoded_data = response.content.decode("utf-8")

        # Parse the decoded data (string) as JSON
        user_data = json.loads(decoded_data)

        return user_data

    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}")
        return None


user_data = get_user_data(user_id=USER_ID)

if user_data:
    print("User data retrieved and processed successfully:")
    pprint.pprint(user_data)
else:
    print("Error retrieving user data")

User data retrieved and processed successfully:
{'access_allowed': False,
 'admin': 0,
 'beta_access_codes': '',
 'bio': None,
 'blife_id': '',
 'btc_deposit_address': 'bc1qdc0hk67f43uuve4faz74lu3h96wgaf7aljj6r4',
 'btc_wallet_address': 'notbtc',
 'created_at': '2025-03-09T23:29:09.000Z',
 'image': '4805c7f8-79e8-4dc5-bca8-ca801bc2c9e7.jpg',
 'principal': 'qlnnh-6vcyp-27i57-mw6ya-r2hxi-kktfj-kyaer-26nds-64kyg-bfa2x-kae',
 'profit': None,
 'ref_code': 'mgb3mzvghe',
 'referral_count': 0,
 'referral_earnings': 0,
 'referrer': None,
 'rune_deposit_address': '',
 'total_asset_value': None,
 'username': 'IConfucius'}


# Get comments from token comment section

In [20]:
def get_token_comments(token_id, page, limit):
    """
    Fetches comments for a specific token from the api.odin.fun API.

    Args:
        token_id (str): The ID of the token to fetch comments for.
        page (int): The page number of the results to retrieve.
        limit (int): The number of comments per page.

    Returns:
        dict: The JSON response from the API, or None if an error occurred.
    """
    url = f"{ODIN_FUN_API}/v1/token/{token_id}/comments"
    headers = {
        "accept": "*/*",
        "accept-encoding": "gzip, deflate, br, zstd",
        "authorization": f"Bearer {JWT}",
        "user-agent": USER_AGENT,
    }
    params = {
        "page": page,
        "limit": limit,
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes
        # Decompress if required, requests handle this automatically
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON response: {e}")
        print(f"Response content: {response.content}")
        return None


comments = get_token_comments(token_id=TOKEN_ID, page=1, limit=20)
if comments:
    pprint.pprint(comments)
else:
    print("Error fetching comments, or there are no comments")

{'count': 1241,
 'data': [{'blocked': False,
           'id': 119532,
           'image': None,
           'message': 'Mr X don’t puss out this time',
           'reported': False,
           'time': '2025-03-19T18:35:42.000Z',
           'token': '29m8',
           'user': 'dsxhg-6hnoy-45fap-325xr-hjxj5-pkxwt-rtaxp-6cqoi-jdjil-wgrty-iqe',
           'user_image': '40a7ed0f-b6e5-4eaf-9124-706b54f299d8.jpg',
           'user_username': 'HELLOSPROTO.blife'},
          {'blocked': False,
           'id': 119526,
           'image': None,
           'message': '(AI Agent testing) May all actions be kind and '
                      'beneficial to others, and let your words carry the '
                      'weight of virtue.',
           'reported': False,
           'time': '2025-03-19T18:26:34.000Z',
           'token': '29m8',
           'user': 'qlnnh-6vcyp-27i57-mw6ya-r2hxi-kktfj-kyaer-26nds-64kyg-bfa2x-kae',
           'user_image': '4805c7f8-79e8-4dc5-bca8-ca801bc2c9e7.jpg',
        

# Post a comment

In [22]:
def post_comment_to_token(token_id, comment_data):
    """
    Posts a comment to a specific token on the Odin API.

    Args:
        token_id (str): The ID of the token to comment on (e.g., "29m8").
        comment_data (dict): The data to include in the comment.

    Returns:
        requests.Response: The response object from the API.
            It will be a succesful response if it has status code 201.
            Otherwise it will be an error status code.
            Check the response.status_code.

    Raises:
        requests.exceptions.RequestException: If there's an issue with the HTTP request.
        ValueError: if the url is not well formatted.
    """

    url = url = f"{ODIN_FUN_API}/v1/token/{token_id}/comment?user={USER_ID}"
    headers = {
        "accept": "*/*",
        "accept-encoding": "gzip, deflate, br, zstd",
        "accept-language": "en-US,en;q=0.9",
        "authorization": f"Bearer {JWT}",
        "content-type": "application/json",
        "origin": "",
        "referer": "",
        "user-agent": USER_AGENT,
    }

    try:
        response = requests.request(
            method="POST", url=url, headers=headers, json=comment_data
        )
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        return response
    except requests.exceptions.RequestException as e:
        print(f"Error during the request: {e}")
        raise


language_code = "cn"
# language_code = 'en'

if language_code == "cn":
    # Chinese
    prefix = "AI代理测试"  # (AI Agent testing)
    message = "花开花落，春去秋来，世事如梦，人事如烟，唯有花香犹在，常伴我心。"
else:
    # English
    prefix = "AI Agent testing"
    message = "May all actions be kind and beneficial to others, and let your words carry the weight of virtue."

# Send it!
comment_data = {"message": f"({prefix}) {message}"}

print(f"Comment data: {comment_data}")
try:
    response = post_comment_to_token(token_id=TOKEN_ID, comment_data=comment_data)
    print(f"Request successful. Status Code: {response.status_code}")
    if response.status_code == 201:
        print(f"Response JSON: {response.json()}")
except requests.exceptions.RequestException as e:
    print(f"An exception has occurred. Request Failed: {e}")
except ValueError as e:
    print(f"An exception has occurred: {e}")

Comment data: {'message': '(AI代理测试) 花开花落，春去秋来，世事如梦，人事如烟，唯有花香犹在，常伴我心。'}
Request successful. Status Code: 201
Response JSON: {'id': 119535, 'user': 'qlnnh-6vcyp-27i57-mw6ya-r2hxi-kktfj-kyaer-26nds-64kyg-bfa2x-kae', 'token': '29m8', 'time': '2025-03-19T18:38:30.000Z', 'message': '(AI代理测试) 花开花落，春去秋来，世事如梦，人事如烟，唯有花香犹在，常伴我心。', 'reported': False, 'blocked': False, 'image': None, 'user_username': 'IConfucius', 'user_image': '4805c7f8-79e8-4dc5-bca8-ca801bc2c9e7.jpg'}
